# 異常検知とは
異常検知とはデータセットのデータ傾向とは大きくかけ離れた異質なデータを発見する処理です。  
機械学習では一般的には教師なし学習として扱われますが問題設定においては教師あり学習としても扱われます。  
<font color="Crimson">異常検知では、システムの故障予知やクレジットカードの不正利用検知、迷惑メール判定など様々な分野で活用されます。</font>

### 異常検知の種類
異常検知は、目的に合わせて適切な手法を選択する必要があります。
代表的な手法として、**外れ値検知、異常部位検出、変化点検知**の3つあります。
 
<font color="Crimson">外れ値検知（Outlier detection）とは、統計モデルに基づいてデータを比較し、期待されるパターンとは異なる結果が観測されたときに検知する手法</font>です。    
古典的な手法の1つにホテリング理論と呼ばれる手法があります。  
ホテリング理論は統計モデルに基づいて各データに対する異常度を算出し、異常度がある閾値を超えた場合外れ値であるとみなす手法です。  
他にも機械学習ベースの手法として、One-class SVMやLocal Outlier Factor(LOF/局所外れ値因子法)、Elliptic Envelope、Lsolation Forestなどの手法もあります。
 
<font color="Crimson">異常部位検出とは、これまでとは違う動きが発生した場合に異常がある部分時系列を検出する目的で使われる手法</font>です。時系列データの異常検知や心電図データから異常部位のみを抜き出したい場合などで応用されます。近傍法などを用いて、部分時系列が異常かどうかを評価します。
 
<font color="Crimson">変化点検知とは、時系列データのパターンが急激に変化する箇所を検知するための手法</font>です。
Webサイトのアクセスの急激に上昇/下落した時期を把握したい場合や特定ワードの検索推移などの異常検知などに利用できます。
ARモデル（自己回帰モデル）やARMAモデル、状態空間モデルを利用する場合もあります。

## 外れ値検知とは
外れ値検知（Outlier detection）とは、統計モデルに基づいてデータを比較し、期待されるパターンとは異なる結果が観測されたときに検知する手法です。
 
この章では外れ値データを混ぜ合わせた擬似データを作成し、正常データと異常データを分離する決定境界線を引くことを目標として進めます。
また同じデータに対してOneClass SVMとIsolation Forestを用いて外れ値検知を行います。
 
まずは下記のコードで外れ値データを含む擬似データを作成します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# make_blobs()は擬似データセットを生成するために使用します。
# centers: 中心の個数（塊数）
# cluster_std:	クラスタの標準偏差
# n_samples:　サンプル数
# n_features: 特徴量の数

data = make_blobs(centers = [[2,2] , [-2, -2]],
                 cluster_std = [0.5, 0.5],
                 random_state = 123, 
                 n_samples = 300,
                 n_features=2)[0]

# random.uniform(low, high)は範囲を指定した乱数を生成します
# low以上high未満の一様乱数を1個生成
# sizeはのタプルで渡します
outliers = np.random.uniform(low = -6, high = 6, size = (int(300 * 0.15),  2))

# concatenate()は複数の配列ndarrayを結合する関数です
# 第二引数axisに結合する軸（次元）を0始まりで指定します。初期値はaxis=0です
X_train = np.concatenate([data, outliers], axis=0)

plt.scatter(data[:, 0], data[:, 1], label="inlier", c='b')
plt.scatter(outliers[:, 0], outliers[:, 1], label="Outlier", c='r')

# locの'upper left'ではアンカーに枠の左上を合わせに設定しています。
plt.legend(loc = 'upper left')

plt.xlim(-10, 10) # x軸方向の表示範囲を取得または指定しています。
plt.ylim(-10, 10) # y軸方向の表示範囲を取得または指定しています。
plt.grid(True) # グリッド（格子）をグラフに表示させる設定です。
plt.show()

### OneClass SVM
先ほど作成した擬似データにOneClass SVMを適用してみます。
OneClass SVMとは機械学習の分類アルゴリズムである サポートベクターマシン (Support Vector Machine:SVM) を教師なしに応用した手法です。  
https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html

In [ ]:
from sklearn.svm import OneClassSVM

# トレーニングエラーの割合の上限とサポートベクトルの割合の下限。
# 間隔（0、1]内にある必要があります。初期値は0.5です。
# kernel: アルゴリズムで使用するカーネルタイプを指定します。今回はrbfを指定しています。
model = OneClassSVM(nu=0.15, kernel='rbf', gamma=0.1)
model.fit(X_train)

## 異常データを等高線で囲みます。

# x, y,・・・の各座標の要素列から格子座標を作成するための関数です。
aa, bb = np.meshgrid(np.linspace(-7, 7, 150),
                     np.linspace(-7, 7, 150))

# numpyのc_は配列を結合する１つです。
# .ravel()ではNumPy配列ndarrayを一次元化（平坦化、flatten）する関数です。
Z = model.predict(np.c_[aa.ravel(), bb.ravel()])
Z = Z.reshape(aa.shape)

plt.scatter(data[:, 0], data[:, 1], label="Inlier", c='b')
plt.scatter(outliers[:, 0], outliers[:, 1], label="Outlier", c='r')

# meshgrid で作った X と Y、そして高さ Z を contour に渡しています。
plt.contour(aa, bb, Z, levels=[0], linewidths=2, colors='red')
plt.legend(loc='upper left')

plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.grid(True)
plt.show()

OneClass SVMは外れ値の影響を受けやすくそのため、そこまでうまく境界線が引けていません。

### Isolation Forest
それでは次にIsolation Forestを用いて境界線を引いてみましょう。  
Isolation Forestとはランダムフォレストを応用したアプローチです。  
まず分割する決定木を複数作成し、ランダムに特徴量を選択し、選択した特徴量の最小値と最大値の範囲に収まるランダムな値を基準にデータを「分離」していきます。そしてこれらを再帰的に繰り返すことにより分離方法を表現する木を複数個作成します。（各木における分割数は木のルートから終端ノードまでのパスの長さに相当します。）  
正常なデータはデータ同士が密集しているため、各データを単一の葉に孤立させるために必要な木の深さが多くなる一方で、外れ値のデータは他の点から離れているため少ない分割数で孤立させられます。  
つまり<font color="Crimson">Isolation Forestはパスの長さが異常度を表す尺度で、各木で作成されたパスの平均長より短いパスで分割されるデータを異常と判断します。</font>


In [ ]:
from sklearn.ensemble import IsolationForest
outlier_ratio = 0.15

model = IsolationForest(contamination=outlier_ratio, random_state=42)
model.fit(X_train)

aa, bb = np.meshgrid(np.linspace(-7, 7, 150),
                     np.linspace(-7, 7, 150))

Z = model.predict(np.c_[aa.ravel(), bb.ravel()])
Z = Z.reshape(aa.shape)

plt.scatter(data[:, 0], data[:, 1], label='Inlier', c='b')
plt.scatter(outliers[:, 0], outliers[:, 1], label='Outlier', c='r')
plt.contour(aa, bb, Z, levels=[0], linewidths=3, colors='red')
plt.legend(loc='upper left')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.grid(True) # グリッド（格子）をグラフに表示させる設定です。
plt.show()

先ほどのOneClass SVMに比べうまく決定境界線が引かれています。